In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir "/content/drive/MYDrive/glove"
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d "/content/drive/MyDrive/glove"

In [ ]:
!export CUDA_VISIBLE_DEVICES=0,1,2,3

In [ ]:
!pip install OpenNMT-py==1.2.0

In [ ]:
!pip install -r /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/requirements.opt.txt

In [ ]:
!onmt_preprocess \
-train_src /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/text_files/src_train.txt \
-train_tgt /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/text_files/tgt_train.txt \
-valid_src /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/text_files/src_dev.txt \
-valid_tgt /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/text_files/tgt_dev.txt \
-save_data /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/demo

In [ ]:
!python /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/tools/embeddings_to_torch.py \
-emb_file_both /content/drive/MyDrive/glove/glove.6B.100d.txt \
-dict_file /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/demo.vocab.pt \
-output_file /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/embeddings

In [ ]:
!onmt_train -save_model /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/model \
           -batch_size 64 \
           -layers 2 \
           -rnn_size 500 \
           -word_vec_size 500 \
           -pre_word_vecs_enc /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/embeddings.enc.pt \
           -pre_word_vecs_dec /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/embeddings.dec.pt \
           -data /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/demo\
           -world_size 1 -gpu_ranks 0 \
           -save_checkpoint_steps 10000 \
           -report_every 5000  


In [ ]:
!onmt_translate -model /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/model_step_100000.pt \
-src /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/text_files/src_test.txt\
 -tgt /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/text_files/tgt_test.txt\
  -output /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/text_files/pred.txt

In [ ]:
!python /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/sql_gen.py \
/content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/text_files/tgt_test.txt \
/content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/wikisql_data/tokenized_test.tables.jsonl \
/content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/text_files/tgt_test_sql.txt 

In [ ]:
!python /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/sql_gen.py \
/content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/text_files/pred.txt \
/content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/wikisql_data/tokenized_test.tables.jsonl \
/content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/text_files/pred_sql.txt 

In [ ]:
!python /content/drive/MyDrive/Text_to_SQL_using_OpenNMT/evaluate_text.py \
/content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/text_files/tgt_test_sql.txt \
/content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/wikisql_data/test.db \
/content/drive/MyDrive/Text_to_SQL_using_OpenNMT/data/text_files/pred_sql.txt